# 1000 Genomes data analysis

First we need to download the 1000 genomes data; we will use the q-arm of chromosome 20. Trees have already been inferred for this data and are available from [Zenodo](https://zenodo.org/record/5495535):


In [169]:
import urllib.request
#urllib.request.urlretrieve("https://zenodo.org/record/5495535/files/hgdp_tgp_sgdp_chr20_q.dated.trees.tsz?download=1",
#                           "data/hgdp_tgp_sgdp_chr20_q.dated.trees.tsz")

I ran `tsunzip data/hgdp_tgp_sgdp_chr20_q.dated.trees.tsz` to unzip, such that we can load the data


In [170]:
import tskit
import numpy as np
import pandas as pd

ts = tskit.load("data/hgdp_tgp_sgdp_chr20_q.dated.trees")
ts


We need to find a trio and randomly sample three individuals. I have the acession numbers for the trio, so we just need a table with nodes and accession ids

In [171]:
import json

meta = ts.individual(0).metadata
print(json.loads(meta))
print(json.loads(meta)['sample'])
str(meta)

{'array_non_reference_discordance': '0.111741', 'capmq': '29', 'coverage': '29.71', 'freemix': '0.00137', 'insert_size_average': '500.1', 'library': 'HGDP01201.11144852', 'library_type': 'PCR', 'region': 'AFRICA', 'sample': 'HGDP01201', 'sample_accession': 'ERS474141', 'sex': 'F', 'source': 'sanger'}
HGDP01201


'b\'{"array_non_reference_discordance": "0.111741", "capmq": "29", "coverage": "29.71", "freemix": "0.00137", "insert_size_average": "500.1", "library": "HGDP01201.11144852", "library_type": "PCR", "region": "AFRICA", "sample": "HGDP01201", "sample_accession": "ERS474141", "sex": "F", "source": "sanger"}\''

That works fine, but some of the samples don't have accession ids:

In [172]:
meta2 = ts.individual(3752).metadata
print(json.loads(meta2))
print(json.loads(meta)['sample'])

{'aliases': 'zapo0098', 'contributor': 'William Klitz / Cheryl Winkler', 'country': 'Mexico', 'dna_source': 'Genomic_from_cell_lines', 'embargo': 'FullyPublic', 'gender': 'M', 'illumina_id': 'LP6005443-DNA_A12', 'region': 'America', 'sample_id': 'zapo0098', 'sequencing_panel': 'C', 'sgdp_id': 'S_Zapotec-1', 'town': 'San Juan Guelavia'}
HGDP01201


So we need to figure out the metadata structure for this data

In [173]:
ts.table_metadata_schemas

TableMetadataSchemas(node=, edge=, site=, mutation=, migration=, individual=, population=)

That's not helpful. Maybe it would be easier to just do regexp on the raw JSON data. Let's figure it out for one sample

In [174]:
import re
bool(re.search("ERS474141", str(meta)))

True

In [175]:
meta

b'{"array_non_reference_discordance": "0.111741", "capmq": "29", "coverage": "29.71", "freemix": "0.00137", "insert_size_average": "500.1", "library": "HGDP01201.11144852", "library_type": "PCR", "region": "AFRICA", "sample": "HGDP01201", "sample_accession": "ERS474141", "sex": "F", "source": "sanger"}'

In [176]:
print(bool(re.search("HGDP01201|dfsdf", str(meta))))
print(bool(re.search("HGDP01202", str(meta))))

True
False


 We can get the Accession IDs for one trio from [this article](https://wikis.utexas.edu/display/bioiteam/Human+Trios+--+GVA2020). Now let's search

In [177]:
trio_ceu = "NA12892|NA12891|NA12878"

d = []
for i in ts.individuals():
    if bool(re.search(trio_ceu, str(i.metadata))):
        d.append(i.id)
d

[2711]

In [178]:
ts.individual(2711)

Individual(id=2711, flags=0, location=array([], dtype=float64), parents=array([], dtype=int32), nodes=array([5422, 5423], dtype=int32), metadata=b'{"family_id": "1463", "gender": "2", "individual_id": "NA12878", "maternal_id": "NA12892", "other_comments": null, "paternal_id": "NA12891", "phenotype": null, "relationship": "mother; child", "second_order": null, "siblings": null, "third_order": null}')

Well this is peculiar. We only seem to have the proband, not the parents! I found [this list](https://genome.ucsc.edu/cgi-bin/hgTrackUi?db=hg38&g=tgpTrios) of all the trios in 1000 Genomes, so let's check them.

In [179]:
trios_ls = [
    ["NA12892", "NA12891", "NA12878"],
    ["NA19678", "NA19675", "NA19679"],
    ["NA19660", "NA19685", "NA19661"],
    ["HG00732", "HG00733", "HG00731"],
    ["HG00657", "HG00702", "HG00656"],
    ["HG02025", "HG02024", "HG02026"],
    ["NA19238", "NA19240", "NA19239"]
] 

def find_nodes(search_str):
    d = []
    for i in ts.individuals():
        if bool(re.search(search_str, str(i.metadata))):
            d.append(str(i.id))
    return ", ".join(d)


trios_df = pd.DataFrame(trios_ls, columns=["mother","child","father"])
trios_df["search_str"]= trios_df.apply(lambda x: '|'.join(x.values.tolist()), axis=1)
trios_df["search_results"] = trios_df["search_str"].apply(lambda x: find_nodes(x))
trios_df


,mother,child,father,search_str,search_results
0,NA12892,NA12891,NA12878,NA12892|NA12891|NA12878,2711
1,NA19678,NA19675,NA19679,NA19678|NA19675|NA19679,"3150, 3151"
2,NA19660,NA19685,NA19661,NA19660|NA19685|NA19661,"3144, 3146"
3,HG00732,HG00733,HG00731,HG00732|HG00733|HG00731,"1238, 1239"
4,HG00657,HG00702,HG00656,HG00657|HG00702|HG00656,"1214, 1215"
5,HG02025,HG02024,HG02026,HG02025|HG02024|HG02026,"1691, 1692"
6,NA19238,NA19240,NA19239,NA19238|NA19240|NA19239,"3045, 3046"


In [180]:
print(json.loads(ts.individual(3150).metadata))
print(json.loads(ts.individual(3151).metadata))

{'family_id': 'm009', 'gender': '2', 'individual_id': 'NA19678', 'maternal_id': None, 'other_comments': None, 'paternal_id': None, 'phenotype': None, 'relationship': 'mother', 'second_order': 'NA19677', 'siblings': None, 'third_order': None}
{'family_id': 'm009', 'gender': '1', 'individual_id': 'NA19679', 'maternal_id': None, 'other_comments': None, 'paternal_id': None, 'phenotype': None, 'relationship': 'father', 'second_order': 'NA19677', 'siblings': None, 'third_order': None}


So for all the other trios besides the first (which is the CEU trio), only the parents are in the TS, not the child. I'm guessing this we usually exclude related individuals when calculating allele frequencies or conducting a GWAS. It seems like this dataset isn't helpful for our purposes.